In [ ]:
import numpy as np
import torch

# (참고용) 이전에 정의했던 함수들이 필요합니다.
# def mixup_data(...): ...
# def cutmix_data(...): ...
# def mixup_criterion(...): ...

def train(model, train_loader, test_loader, epochs, aug_method='none'):
    model.to(device)
    history = {'val_accuracy': []}
    
    print(f"Start training with method: {aug_method}")

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # --- [핵심 수정 부분] Augmentation 적용 로직 ---
            if aug_method == 'mixup':
                # 1. MixUp 데이터 생성
                images, targets_a, targets_b, lam = mixup_data(images, labels, alpha=1.0)
                # 2. 모델 예측
                outputs = model(images)
                # 3. MixUp Loss 계산
                loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                
            elif aug_method == 'cutmix':
                # CutMix는 확률적으로(예: 50%) 적용하거나 항상 적용할 수 있습니다.
                # 여기서는 항상 적용하는 것으로 작성합니다.
                images, targets_a, targets_b, lam = cutmix_data(images, labels, alpha=1.0)
                outputs = model(images)
                loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                
            else:
                # 일반 학습 (Basic)
                outputs = model(images)
                loss = criterion(outputs, labels)
            # ---------------------------------------------

            loss.backward()
            optimizer.step()

            # 정확도 계산 (MixUp/CutMix일 때는 원본 라벨 중 더 큰 비중을 가진 쪽과 비교하거나, 단순히 가장 높은 확률의 클래스로 계산)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Method: {aug_method}, Train Acc: {train_acc:.2f}%")

        # --- 검증 (Validation) ---
        # 검증 데이터에는 MixUp/CutMix를 적용하지 않습니다 (정석)
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_acc = 100. * val_correct / val_total
        history['val_accuracy'].append(val_acc)
        print(f"Validation Accuracy: {val_acc:.2f}%")

    return history

# --- 실행 예시 ---
# 1. MixUp으로 학습
# history_mixup = train(resnet50, train_loader, test_loader, EPOCH, aug_method='mixup')

# 2. CutMix로 학습
# history_cutmix = train(resnet50, train_loader, test_loader, EPOCH, aug_method='cutmix')